# OPS-SAT case starter-kit notebook

ESA's [Kelvins](https://kelvins.esa.int) competition "[the OPS-SAT case](https://kelvins.esa.int/opssat/home/)" is a novel data-centric challenge that asks you to work with the raw data of a satellite and very few provided labels to find the best parameters for a given machine learning model. Compared to previous competitions on Kelvins (like the [Pose Estimation](https://kelvins.esa.int/pose-estimation-2021/) or the [Proba-V Super-resolution challenge](https://kelvins.esa.int/proba-v-super-resolution/)) where the test-set is provided and the infered results are submitted, for the OPS-SAT case, we will run inference on the Kelvins server directly! This notebooks contains examples on how you can load your data and train an **EfficientNetLite0** model by only using the 80-labeled images provided. Therefore, the directory `images`, containing unlabeld patches and included in the training dataset is not used for this notebook. However, competitors are encouraged to use these patches to improve the model accuracy.

# 1. Module imports

If you do not have a GPU, uncomment and run the next commands.


In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
file = '/home/phillip/PycharmProjects/the_opssat_case_starter_kit'

Other imports.

In [2]:
import tensorflow as tf
import datetime
from tensorflow import keras

import numpy as np
from sklearn.metrics import cohen_kappa_score

from efficientnet_lite import EfficientNetLiteB0

2022-10-17 17:41:20.120848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-17 17:41:20.120883: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# 2. Utility Functions

You can use this function to load your training data.

In [3]:
def get_images_from_path(dataset_path, split: bool = True, _seed: int = 142):
    """ Get images from path and normalize them applying channel-level normalization. """

    # loading all images in one large batch
    if split:
        tf_train_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], shuffle=False, validation_split=0.2, subset="training", seed=_seed)
        tf_eval_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], shuffle=False, validation_split=0.2, subset="validation", seed=_seed)
    
        # extract images and targets
        for tf_eval_images, tf_eval_targets in tf_eval_data:
            break

        for tf_train_images, tf_train_targets in tf_train_data:
            break

        return tf.convert_to_tensor(tf_train_images), tf_train_targets, tf.convert_to_tensor(tf_eval_images), tf_eval_targets, tf_eval_data
    else:
        # loading all images in one large batch
        tf_eval_data = tf.keras.utils.image_dataset_from_directory(dataset_path, image_size=input_shape[:2], shuffle=False, 
                                                                   batch_size=100000)

        # extract images and targets
        for tf_eval_images, tf_eval_targets in tf_eval_data:
            break

        return tf.convert_to_tensor(tf_eval_images), tf_eval_targets

# 3. Loading the model

The network architecture used for OPS-SAT is **EfficientNetLite0**. We would like to thank Sebastian for making a Keras implementation of EfficientNetLite publicly available under the Apache 2.0 License: https://github.com/sebastian-sz/efficientnet-lite-keras. Our Version of this code has been modified to better fit our purposes. For example, we removed the ReLU "stem_activation" to better match a related efficientnet pytorch implementation. In any way, **you have to use the model architecture that we provide in our [starter-kit](https://gitlab.com/EuropeanSpaceAgency/the_opssat_case_starter_kit).**

In [4]:
input_shape = (200, 200, 3)   # input_shape is (height, width, number of channels) for images
num_classes = 8
model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
# model.summary()

2022-10-17 17:41:22.475094: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-17 17:41:22.475131: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-17 17:41:22.475159: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (laptop): /proc/driver/nvidia/version does not exist
2022-10-17 17:41:22.475466: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same


# 4. Loading data

Uncomment next line and adjust with the path of your dataset. 

In [5]:
training_dataset_path=os.path.join(file, 'ops_sat_competiton_official_training')

In this notebook, classical supervised learning is used. Therefore, remember to remove the subdirectory `images` containing unlabeled patches before loading the dataset to perform training correctly.

In [6]:
#Loading dataset
x_train, y_train=get_images_from_path(training_dataset_path, split=False)

Found 746 files belonging to 8 classes.


2022-10-17 17:41:24.301058: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 358080000 exceeds 10% of free system memory.


# 5. Model training

We provide now an example on how you can train your model by using standard supervised learning. Training loss (`SparseCategoricalCrossentropy`) and `Accuracy` are shown for simplicity and for an easier interpretation of the training outcome, despite your submission will be evaluated by using the metric **1 - Cohen's kappa** [metric](https://en.wikipedia.org/wiki/Cohen's_kappa). For more information on scoring, please refer to [Scoring](https://kelvins.esa.int/opssat/scoring/).

In [7]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.SparseCategoricalAccuracy()])

With this model and the dataset provided, please do your best!

In [8]:
# load data, data augmentation, training, overfitting, transfer-learning etc.
history=model.fit(x_train, y_train, epochs=40, verbose=1, batch_size=8)

Epoch 1/40
94/94 [==============================] - 71s 689ms/step - loss: 3.9114 - sparse_categorical_accuracy: 0.2131
Epoch 2/40
94/94 [==============================] - 70s 749ms/step - loss: 3.0040 - sparse_categorical_accuracy: 0.2198
Epoch 3/40
94/94 [==============================] - 72s 766ms/step - loss: 2.6798 - sparse_categorical_accuracy: 0.2708
Epoch 4/40
94/94 [==============================] - 73s 780ms/step - loss: 2.3865 - sparse_categorical_accuracy: 0.2962
Epoch 5/40
94/94 [==============================] - 75s 800ms/step - loss: 2.2368 - sparse_categorical_accuracy: 0.3231
Epoch 6/40
94/94 [==============================] - 75s 797ms/step - loss: 2.1368 - sparse_categorical_accuracy: 0.3164
Epoch 7/40
94/94 [==============================] - 76s 808ms/step - loss: 1.9568 - sparse_categorical_accuracy: 0.3324
Epoch 8/40
94/94 [==============================] - 77s 815ms/step - loss: 1.8431 - sparse_categorical_accuracy: 0.3740
Epoch 9/40
94/94 [======================

Calculating the **1 - Cohen's kappa** score of the trained model on the trained dataset.

In [9]:
def evaulate_on():
    predictions = np.zeros(len(y_eval), dtype=np.int8)

    # inference loop
    for e, (image, target) in enumerate(zip(x_eval, y_eval)):
        image = np.expand_dims(np.array(image), axis=0)
        output = model.predict(image)
        predictions[e] = np.squeeze(output).argmax()

    #Keras model score
    score_keras = 1 - cohen_kappa_score(y_eval.numpy(), predictions)
    print(y_eval.numpy(), predictions)
    print("Score:",score_keras)

# 6. Saving and loading trained model

The trained model can be now saved by using HDF5-format that is the only accepted for submission. The name `test.h5` will be used.

In [10]:
#Saving model
h5_filename = f'test_{datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")}.h5'
model.save_weights(h5_filename)

The trained model can be also loaded for further testing. 

In [11]:
model = EfficientNetLiteB0(classes=num_classes, weights=None, input_shape=input_shape, classifier_activation=None)
model.load_weights(h5_filename)

block1a_ same
block2a_ ((1, 1), (1, 1))
block2a_ valid
block2b_ same
block3a_ ((2, 2), (2, 2))
block3a_ valid
block3b_ same
block4a_ ((1, 1), (1, 1))
block4a_ valid
block4b_ same
block4c_ same
block5a_ same
block5b_ same
block5c_ same
block6a_ ((2, 2), (2, 2))
block6a_ valid
block6b_ same
block6c_ same
block6d_ same
block7a_ same


The model will be now compiled and tested again. You should get the same score as before saving and loading. 

In [12]:
#Model shall be compiled before testing.
model.compile()

#Creating empty predictions
predictions = np.zeros(len(y_train), dtype=np.int8)

# inference loop
for e, (image, target) in enumerate(zip(x_train, y_train)):
    image = np.expand_dims(np.array(image), axis=0)
    output = model.predict(image)
    predictions[e] = np.squeeze(output).argmax()

#Keras model score
score_keras = 1 - cohen_kappa_score(y_train.numpy(), predictions)
print("Score:",score_keras)

Score: 0.4607232729419548
